In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Prev_application 데이터 세트 기반의 EDA와 Feature Engineering 수행 후 학습 모델 생성/평가

In [ ]:
import numpy as np
import pandas as pd
import gc
import time

%matplotlib inline

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

새로운 Feature 만들어서 application_train과 application_test에 추가

In [ ]:
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')

In [ ]:
#app_train과 app_test를 함께 가공해야 하기 때문에 concat 이용해 통합

apps = pd.concat([app_train, app_test])


In [ ]:
#shape 확인하기 : app_train에 column 1개가 더 많음
print('Size of application_train data', app_train.shape)
print('Size of application_test data', app_test.shape)
print('Size of apps', apps.shape)

In [ ]:
apps.head()

In [ ]:
def get_apps_processed(apps):
    
    # EXT_SOURCE_X FEATURE 가공
    apps['APPS_EXT_SOURCE_MEAN'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    apps['APPS_EXT_SOURCE_STD'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    apps['APPS_EXT_SOURCE_STD'] = apps['APPS_EXT_SOURCE_STD'].fillna(apps['APPS_EXT_SOURCE_STD'].mean())
    
    # AMT_CREDIT 비율로 Feature 가공
    apps['APPS_ANNUITY_CREDIT_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_CREDIT']
    apps['APPS_GOODS_CREDIT_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_CREDIT']
    
    # AMT_INCOME_TOTAL 비율로 대출 금액 관련 Feature 가공
    apps['APPS_ANNUITY_INCOME_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_INCOME_TOTAL']
    apps['APPS_CREDIT_INCOME_RATIO'] = apps['AMT_CREDIT']/apps['AMT_INCOME_TOTAL']
    apps['APPS_GOODS_INCOME_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_INCOME_TOTAL']
    apps['APPS_CNT_FAM_INCOME_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['CNT_FAM_MEMBERS']
    
    # DAYS_BIRTH, DAYS_EMPLOYED 비율로 소득/자산 관련 Feature 가공
    apps['APPS_EMPLOYED_BIRTH_RATIO'] = apps['DAYS_EMPLOYED']/apps['DAYS_BIRTH']
    apps['APPS_INCOME_EMPLOYED_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_EMPLOYED']
    apps['APPS_INCOME_BIRTH_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_BIRTH']
    apps['APPS_CAR_BIRTH_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_BIRTH']
    apps['APPS_CAR_EMPLOYED_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_EMPLOYED']
    
    return apps

In [ ]:
get_apps_processed(apps)

Previous_application 로딩

In [ ]:
prev_app = pd.read_csv('../input/home-credit-default-risk/previous_application.csv')
print(prev_app.shape, apps.shape)

In [ ]:
prev_app.head()

appication과 previous 조인하고 비교

-누락된 집합 확인

In [ ]:
#appication과 previous 조인(merge)

prev_app_outer = prev_app.merge(apps['SK_ID_CURR'], on='SK_ID_CURR', how='outer', indicator=True)

In [ ]:
prev_app_outer.head()

In [ ]:
prev_app_outer['_merge'].value_counts()

In [ ]:
# Null 값을 확인하고 전체 데이터에서 비율을 보여주는 함수

def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [ ]:
prev_app.info()

In [ ]:
missing_data(prev_app).head(30)

Target 유형에 따라 숫자형 feature 확인

In [ ]:
# prev_app와 app_train을 inner 조인 ==> TARGET 값은 application_train(app_train) 데이터에만 있음
# ==> prev_app안에 TARGET 유형에 따라 FEATURE 확인을 하려면 조인할 필요가 있음

# app_train[['SK_ID_CURR', 'TARGET']] 두가지 컬럼만 갖고 옴
# on='SK_ID_CURR' 컬럼 기준으로 조인

app_prev_target = prev_app.merge(app_train[['SK_ID_CURR', 'TARGET']], on='SK_ID_CURR', how='left')
app_prev_target.shape

In [ ]:
# TARGET 1 일때 값을 확인
# TARGET 0 일때 값을 확인

def show_hist_by_target(df, columns):
    cond_1 = (df['TARGET'] == 1)
    cond_0 = (df['TARGET'] == 0)
    
    for column in columns:
        fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 4), squeeze=False)
        sns.violinplot(x='TARGET', y=column, data=df, ax=axs[0][0] )
        sns.distplot(df[cond_0][column], ax=axs[0][1], label='0', color='blue')
        sns.distplot(df[cond_1][column], ax=axs[0][1], label='1', color='red')   

In [ ]:
# Numerical Features(숫자형 피처) 과 Categorical Features(범주/object 피처) 구분 함수

def type_features(data):
    categorical_features = data.select_dtypes(include = ["object"]).columns
    numerical_features = data.select_dtypes(exclude = ["object"]).columns
    print( "categorical_features :",categorical_features)
    print('-----'*40)
    print("numerical_features:",numerical_features)

In [ ]:
type_features(app_prev_target)

Numerical Features & Categorical Feature 위한 시각화 함수

In [ ]:
# 시각화 함수
def plot_re(df,t1='',t2=''):
    f,ax=plt.subplots(1,2,figsize=(12,8))
    df[[t1,t2]].groupby([t1]).count().plot.bar(ax=ax[0],color='Blue')
    ax[0].set_title('count of customer on '+t1)
    sns.countplot(t1,hue=t2,data=df,ax=ax[1],palette="spring")
    ax[1].set_title(t1+': Target 0 vs Target 1')
    # Rotate x-labels
    plt.xticks(rotation=-90)
    a=plt.show()
    return a

previous_application을 이용한 EDA

In [ ]:
#함수 호출
#대출 유형에 따른 target

plot_re(app_prev_target, 'NAME_CONTRACT_TYPE','TARGET')

#대부분 CASH LANS과 CONSUMER LOANS에 몰려 있으며, CASH LOASNS에서는 CONSUMER LOASNS보다 1 TARGET의 비율이 높음

In [ ]:
#동행 고객에 따른 TARGET

plot_re(app_prev_target, 'NAME_TYPE_SUITE','TARGET')

#동행이 없는 사람이 가장 많으며, 동행이 있는 사람들 중에는 가족이 가장 많음.
#혼자 온 사람들의 대부분은 TARGET 1이고 가족과 함께 온 사람들의 TARGET 1 비율과 비슷해 보임

In [ ]:
#영업 대출 여부에 따른 TARGET

plot_re(app_prev_target, 'NAME_PRODUCT_TYPE','TARGET')

#XHA가 가장 많으며,WALK-IN 보다 X-SELL이 더 많고, WALK-IN의 TRAGET 1 비율이 더 커보임

In [ ]:
pd.crosstab(app_prev_target.TARGET, app_prev_target.NAME_PRODUCT_TYPE, dropna=False, normalize='all')

numberic feature 시각화

In [ ]:
f, ax = plt.subplots(1,3,figsize=(20, 10))


# RATE_DOWN_PAYMENT : 선금 비율(정규화됨)
# 비율이 대부분 0.0~0.1 사이에 모여있음
sns.distplot(app_prev_target.RATE_DOWN_PAYMENT.dropna(), kde=True, color="g", 
             ax=ax[0]).set_title('RATE DOWN PAYMENT Distribution')

# RATE_INTEREST_PRIMARY : 이자율
sns.distplot(app_prev_target.RATE_INTEREST_PRIMARY.dropna(), kde=True, color="b",
             ax=ax[1]).set_title('RATE INTEREST PRIMARY Distribution')


# RATE_INTEREST_PRIVILEGED : 이자율
sns.distplot(app_prev_target.RATE_INTEREST_PRIVILEGED.dropna(), kde=True, 
             color="r", ax=ax[2]).set_title('RATE INTEREST PRIVILEGED Distribution')

#RATE INTERSET PRIMARY 보다 PRIVILEGED가 비교 했을 때 큰 이자율을 갖는 것을 알 수 있음

In [ ]:
f, ax = plt.subplots(1,3,figsize=(20, 10))

# CNT_PAYMENT : 이전 대출 신청의 대출금액 관련 TERM
# 다양한 분포가 있지만, 0~20 사이의 값 위주로 분포되어 있음
sns.distplot(app_prev_target.CNT_PAYMENT.dropna(), kde=True, color="g", 
             ax=ax[0]).set_title('CNT_PAYMENT Distribution')


# DAYS_FIRST_DUE : 신청 날짜로 부터 마감일까지의 일자
sns.distplot(app_prev_target.DAYS_FIRST_DUE.dropna(), kde=True, color="b",
             ax=ax[1]).set_title('DAYS_FIRST_DUE Distribution')

 
# DAYS_LAST_DUE : 신청 날짜로부터 마지막 만기일까지의 일
sns.distplot(app_prev_target.DAYS_LAST_DUE.dropna(), kde=True, 
             color="r", ax=ax[2]).set_title('DAYS_LAST_DUE Distribution')
#뒤 두 그래프에서는 값이 0 주변에 분포했는데, 이를 통해서 자세히 알 수는 없을 것

In [ ]:
# 'AMT_ANNUITY' : 월 대출 납입액

app_prev_target.groupby(['NAME_CONTRACT_TYPE','NAME_PRODUCT_TYPE']).agg({'AMT_ANNUITY': ['mean', 'median', 'count', 'max']})
# 대출 유형과 영업 대출 여부에 따라 분류 해 월 대출 납입액 확인

In [ ]:
# NAME_CASH_LOAN_PURPOSE : 현금 대출 목적

app_prev_target.groupby(['NAME_CONTRACT_TYPE','NAME_CASH_LOAN_PURPOSE']).agg({'AMT_ANNUITY': ['mean', 'median', 'count', 'max']})

APP_PREV 데이터 세트 FEATURE ENGINEERING 수행

In [ ]:
# groupby SK_ID_CURR
# 
prev_app.groupby('SK_ID_CURR')

In [ ]:
# 변수로 받아서 prev_group에 저장
prev_group = prev_app.groupby('SK_ID_CURR') # groupby 대상 컬럼명
prev_group.head()

만기일까지 월 납입해야 하는 원금과 그 비율은?

In [ ]:
# 대출 금액 AMT_CREDIT에서 전체 납부해야 하는 횟수 CNT_PAYMENT를 나눔
prev_app['MONTH_AMT']=prev_app['AMT_CREDIT']/prev_app['CNT_PAYMENT']
prev_app

In [ ]:
# AMT_ANNUITY에서 원금의 비율 확인
prev_app['RATE_MONTHS_AMT']=100*prev_app['MONTH_AMT']/prev_app['AMT_ANNUITY']

In [ ]:
prev_app

In [ ]:
prev_group.head()

In [ ]:
# SK_IP_CURR을 이용해 각 대출 횟수를 구하고, 월별 원금 납입액의 평균, 최대, 최소 값을 구함
prev_agg = pd.DataFrame()
prev_agg['CNT'] = prev_group['SK_ID_CURR'].count()
prev_agg['AVG_MONTH_AMT']=prev_app.groupby('SK_ID_CURR')['MONTH_AMT'].mean()
prev_agg['MAX_MONTH_AMT']=prev_app.groupby('SK_ID_CURR')['MONTH_AMT'].max()
prev_agg['MIN_MONTH_AMT']=prev_app.groupby('SK_ID_CURR')['MONTH_AMT'].min()

In [ ]:
# 원금 비율에 대한 평균, 최대, 최소 값을 구함
prev_agg['AVG_RATE_MONTHS_AMT']=prev_app.groupby('SK_ID_CURR')['RATE_MONTHS_AMT'].mean()
prev_agg['MAX_RATE_MONTHS_AMT']=prev_app.groupby('SK_ID_CURR')['RATE_MONTHS_AMT'].max()
prev_agg['MIN_RATE_MONTH_AMT']=prev_app.groupby('SK_ID_CURR')['RATE_MONTHS_AMT'].min()

In [ ]:
prev_agg.head()

COUSTOMER 당 대출 상태(허가, 취소) 비율

In [ ]:
prev_app.shape

In [ ]:
prev_app['STATE_APPROVED_REFUSED']=prev_app['NAME_CONTRACT_STATUS']

# Refused 인 경우에는 y에는 0, Approved 인경우에는 Y=1 
def f(x):
    if x == 'Refused':
        y = 0
    else:
        y = 1    
    return y

prev_app['STATE_APPROVED_REFUSED'] = prev_app.apply(lambda x:f(x.NAME_CONTRACT_STATUS), axis=1)


In [ ]:
prev_app

In [ ]:
# COUSTOMER당 APPROVED(=1)인 건수를 합함
prev_app.groupby('SK_ID_CURR')['STATE_APPROVED_REFUSED'].sum()

In [ ]:
# COUSTOMER의 대출 신청 횟수로 APPROVED 건수를 나눠서 승인된 비율 확인
prev_agg['RATE_APPLOVED']=prev_app.groupby('SK_ID_CURR')['STATE_APPROVED_REFUSED'].sum()/prev_agg['CNT']

In [ ]:
prev_agg

In [ ]:
# COUSTOMER 당 대출 승인 비율 분포표
sns.distplot(prev_agg.RATE_APPLOVED.dropna(), kde=True, 
             color="r").set_title('RATE_APPLOVED Distribution')

In [ ]:
# PURPOSE에 따른 승인 비율 분석 추가
#prev_app.groupby('NAME_CASH_LOAN_PURPOSE')['NAME_CASH_LOAN_PURPOSE'].count()
prev_purpose=pd.DataFrame()
prev_purpose['RATE_PURPOSE_APPROVED']=prev_app.groupby('NAME_CASH_LOAN_PURPOSE')['STATE_APPROVED_REFUSED'].sum()/prev_app.groupby('NAME_CASH_LOAN_PURPOSE')['NAME_CASH_LOAN_PURPOSE'].count()

In [ ]:
# 시각회

sns.stripplot(x="RATE_PURPOSE_APPROVED", y=prev_purpose.index, data=prev_purpose)
plt.show()

# XNA(목적 데이터 X), XNP(CASH LOAN이 아닐때)를 제외하고 ELECTONIC EQUIPMENT의 목적에서 승인 비율이 높음

SK_ID_CURR 별 NAME_CONTRACT_STATUS가 REFUSED일 경우 대출 신청 금액

In [ ]:
prev_app['STATE_APPROVED_REFUSED'].value_counts()

In [ ]:
#STATE_APPROVED_REFUSED가 0일 때를 기준으로 filtering
cond_refused = (prev_app['STATE_APPROVED_REFUSED'] == 0)
prev_refused=prev_app[cond_refused]
prev_refused.shape

In [ ]:
# dataframe을 만들어서 거절된 대출 신청의 대출신청 금액을 max와 min을 통해 확인
prev_refused_group=pd.DataFrame()
prev_refused_group['REFUSED_COUNTS']=prev_refused.groupby('SK_ID_CURR')['SK_ID_CURR'].count()
prev_refused_group['MAX_REFUSED_AMT_APP']=prev_refused.groupby('SK_ID_CURR')['AMT_APPLICATION'].max()
prev_refused_group['MIN_REFUSED_AMT_APP']=prev_refused.groupby('SK_ID_CURR')['AMT_APPLICATION'].min()

In [ ]:
prev_refused_group

In [ ]:
# merge를 위해 두 dataframe에 reset_index
prev_refused_group=prev_refused_group.reset_index()
prev_agg_reset=prev_agg.reset_index()

In [ ]:
# merge
prev_refused_agg = prev_agg.merge(prev_refused_group, on='SK_ID_CURR', how='left')

In [ ]:
# Null값을 0으로 변경
prev_refused_agg=prev_refused_agg.fillna(0)

In [ ]:
prev_refused_agg.head()

한 사람당 승인된 비율

In [ ]:
prev_refused_agg['PREV_APPROVED_RATIO'] =1- prev_refused_agg['REFUSED_COUNTS'] / prev_refused_agg['CNT']

In [ ]:
prev_refused_agg.head(10)

승인된 신청 금액과 대출금액 차이

In [ ]:
#STATE_APPROVED_REFUSED가 1일 때를 기준으로 filtering
cond_approved = (prev_app['STATE_APPROVED_REFUSED'] == 1)
prev_approved=prev_app[cond_approved]
prev_approved.shape

In [ ]:
# 허가된 대출 금액에서 신청된 대출 금액을 빼어 둘의 차이를 구함
prev_approved['DIFF_APPLICATION_CREDIT']=prev_approved['AMT_CREDIT']-prev_approved['AMT_APPLICATION']

In [ ]:
# 신청 금액에서 몇 % 변화가 있었는지
prev_approved['CHANGE_CREDIT_DIFF']=100*prev_approved['DIFF_APPLICATION_CREDIT']/prev_approved['AMT_APPLICATION']

In [ ]:
# SK_ID_CURR의 MAX,MIN DIFFERENCE를 가져옴
prev_approved_group=pd.DataFrame()
prev_approved_group['MAX_REFUSED_AMT_APP']=prev_approved.groupby('SK_ID_CURR')['CHANGE_CREDIT_DIFF'].max()
prev_approved_group['MIN_REFUSED_AMT_APP']=prev_approved.groupby('SK_ID_CURR')['CHANGE_CREDIT_DIFF'].min()

In [ ]:
# RESET_INDEX하고 prev_refused_agg와 MERGE
prev_approved_reset= prev_approved_group.reset_index()
prev_refused_approved_agg = prev_refused_agg.merge(prev_approved_reset, on='SK_ID_CURR', how='left')

In [ ]:
# Null값을 0으로 변경
prev_refused_approved_agg=prev_refused_approved_agg.fillna(0)

In [ ]:
prev_refused_approved_agg

In [ ]:
apps_all= get_apps_processed(apps)

In [ ]:
prev_agg

In [ ]:
print(apps_all.shape, prev_refused_approved_agg.shape)
apps_all=apps_all.merge(prev_refused_approved_agg, on='SK_ID_CURR', how='left')
print(apps_all.shape)

In [ ]:
apps_all.info()

데이터 레이블 인코딩, NULL값은 LightGBM 내부에서 처리하도록 특별한 변경하지 않음

In [ ]:
object_columns = apps_all.dtypes[apps_all.dtypes == 'object'].index.tolist()
for column in object_columns:
    apps_all[column] = pd.factorize(apps_all[column])[0]

In [ ]:
apps_all.info()

학습 데이터와 테스트 데이터 다시 분리

In [ ]:
apps_all_train = apps_all[~apps_all['TARGET'].isnull()]
apps_all_test = apps_all[apps_all['TARGET'].isnull()]

apps_all_test = apps_all_test.drop('TARGET', axis=1)

In [ ]:
apps_all_train.columns.tolist()

학습 데이터를 검증 데이터로 분리하고 LGBM Classifier로 학습 수행

In [ ]:
from sklearn.model_selection import train_test_split

ftr_app = apps_all_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
target_app = apps_all_train['TARGET']

train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
train_x.shape, valid_x.shape

In [ ]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
        early_stopping_rounds= 50)

In [ ]:
from lightgbm import plot_importance

plot_importance(clf, figsize=(16, 32))

학습된 Classifier를 이용하여 테스트 데이터 예측하고 결과를 Kaggle로 Submit 수행

In [ ]:
preds = clf.predict_proba(apps_all_test.drop('SK_ID_CURR', axis=1))[:, 1 ]
apps_all_test['TARGET'] = preds
apps_all_test[['SK_ID_CURR', 'TARGET']].to_csv('prev_je2760sul.csv', index=False)